In [1]:
import sys
# sys.path.append("/home/jupyter-wzr/.local/lib/python3.8/site-packages")
import warnings
import sklearn
from sklearn import preprocessing
import torch.nn.functional as F
import torch
from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from torch import nn
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
# 其实线性回归就是一个不加激活函数的全链接层
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import glob,os
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" # 设置可⻅GPU，从0开始

/home/jupyter-wzr/.conda/envs/torch/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 数据读取与处理

In [2]:
filenames= []
for m in range (10):
    for n in range(m+1,11):        
        strm = 'soc_start_'+str(m)
        strn = 'soc_end_'+str(n)
        path=r'/home/jupyter-wzr/0927/0929data/'+strm+'/'+strn
        filename=glob.glob(os.path.join(path, "*.csv"))
        filenames.extend(filename)

dl = []
for f in filenames:
    dl.append(pd.read_csv(f, header=0, sep=','))
df = pd.concat(dl).loc[:, ['set_P','SOC_start','SOC_end','set_SOH','delta_SOH']]
df.iloc[:,0].size

874668

In [3]:
zscores = preprocessing.StandardScaler()
# zscore标准化
zscore = zscores.fit_transform(df)
df_zscore = pd.DataFrame(zscore,index=df.index,columns = df.columns)#df.columns
df_comp = sklearn.utils.shuffle(df_zscore)
df_comp.head(10)

,set_P,SOC_start,SOC_end,set_SOH,delta_SOH
282,-1.397442,-0.175789,-0.340301,1.151305,0.262872
1651,1.016128,-0.628658,-0.645394,-0.040953,0.493682
16332,-0.432014,-0.830430,-0.951360,-1.512175,0.584586
95,0.211604,0.828839,0.576933,1.382981,-0.162997
949,1.820651,2.168491,1.681589,0.548350,0.424210
1016,1.659746,-0.810741,-0.950812,1.539562,0.573648
17943,-1.236538,-0.774468,-0.951492,-0.048443,0.600045
3,0.372509,1.163521,1.491877,1.851919,-1.705633
1647,1.177032,-0.512019,-0.645339,-0.310336,0.460097
47559,-1.236538,-0.737071,-0.951477,-1.456587,0.604934


In [4]:
# 选择变量
y=df_comp['delta_SOH']
x=df_comp[['set_P','SOC_start','SOC_end','set_SOH']]

# 切分训练集与测试集与验证集
x_train_all, x_test, y_train_all, y_test = train_test_split(x,y, test_size=0.2)  # 不要设置随机种子random_state保证每次划分的随机性
x_train, x_val, y_train, y_val = train_test_split(x,y, test_size=0.5)  # 不要设置随机种子random_state保证每次划分的随机性


In [5]:
x_train, x_val, y_train, y_val = np.array(x_train), np.array(x_val), np.array(y_train), np.array(y_val)

In [6]:
x_train, x_val, y_train, y_val = \
torch.tensor(x_train), torch.tensor(x_val), torch.tensor(y_train), torch.tensor(y_val)

In [7]:
len(x_train),len(y_train)

(437334, 437334)

# ICNN模型建立

In [14]:
class CvxModel(nn.Module):
    '''
        输入凸神经网络结构定义
    '''

    def __init__(self, n_feature=4, n_hidden=4, n_output=1, nGdIter=30):
        super(CvxModel, self).__init__()
        self.input_layer = nn.Linear(n_feature, n_hidden, bias=False)
        self.hidden_layer = nn.Linear(n_hidden, n_hidden, bias=False)
        self.output_layer = nn.Linear(n_hidden, n_output, bias=False)
        self.passthrough_layer = nn.Linear(n_feature, n_hidden)
        self.passthrough_output_layer = nn.Linear(n_feature, n_output)

    def forward(self, x):
        x = x.to(torch.float32)
        zx1 = F.relu(self.input_layer(x))
        pass1 = self.passthrough_layer(x)
        pass2 = self.passthrough_output_layer(x)
        zx2 = F.relu(self.hidden_layer(zx1) + pass1)
        zx3 = self.output_layer(zx2) + pass2
        zx3 = zx3.squeeze(-1)
        return zx3


class weightConstraint():
    '''
        对一个特定的层的weight进行参数限制
    '''

    def __init__(self):
        pass

    def __call__(self, module):
        if hasattr(module, 'weight'):
            w = module.weight.data
            w = w.clamp(0.0, 100.0) #clamp函数可以设置最大最小值，
            module.weight.data = w


# 当对权重有非负的限制时，训练神经网络
def train_with_constrains(): 
    model = CvxModel(n_feature=4, n_hidden=200, n_output=1)
    constraints = weightConstraint()
    criterion = torch.nn.MSELoss(reduction='sum')
    learning_rate = 1e-3
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    terminate_threshold = 1e-3 # 1e-5
    i = 0
    print("当前程序开始运行时间为：{}".format(time.asctime(time.localtime(time.time()))))
    while (True):
        
        y_pred = model(x_train)
        loss = (criterion((y_pred).float(), (y_train).float()) / y_train.size()[0])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        model._modules['input_layer'].apply(constraints)
        model._modules['hidden_layer'].apply(constraints)
        model._modules['output_layer'].apply(constraints)
        i += 1
        if i % 100 == 0:
            print("当前程序运行结束时间为：{}".format(time.asctime(time.localtime(time.time()))))
            print("当前正在训练第{}次， 损失是{}".format(i, loss.item()))
#             print(y_train-y_pred)
            # test_model(model)
            # weight_print(model)
        if loss.item() < terminate_threshold:
            print("训练满足精度要求，训练了{}次。".format(i))
            break

def test_model(model):
    print(model.parameters)



In [15]:
train_with_constrains()


当前程序开始运行时间为：Thu Oct 13 10:28:51 2022
当前程序运行结束时间为：Thu Oct 13 10:30:10 2022
当前正在训练第100次， 损失是1.5153485536575317
当前程序运行结束时间为：Thu Oct 13 10:31:27 2022
当前正在训练第200次， 损失是1.1870265007019043
当前程序运行结束时间为：Thu Oct 13 10:32:46 2022
当前正在训练第300次， 损失是0.9968810081481934
当前程序运行结束时间为：Thu Oct 13 10:34:05 2022
当前正在训练第400次， 损失是0.859103798866272
当前程序运行结束时间为：Thu Oct 13 10:35:25 2022
当前正在训练第500次， 损失是0.759321391582489
当前程序运行结束时间为：Thu Oct 13 10:36:43 2022
当前正在训练第600次， 损失是0.6864757537841797
当前程序运行结束时间为：Thu Oct 13 10:37:59 2022
当前正在训练第700次， 损失是0.6311678886413574
当前程序运行结束时间为：Thu Oct 13 10:39:17 2022
当前正在训练第800次， 损失是0.5848026871681213
当前程序运行结束时间为：Thu Oct 13 10:40:37 2022
当前正在训练第900次， 损失是0.5387942790985107
当前程序运行结束时间为：Thu Oct 13 10:42:00 2022
当前正在训练第1000次， 损失是0.47864997386932373
当前程序运行结束时间为：Thu Oct 13 10:43:20 2022
当前正在训练第1100次， 损失是0.4168975353240967
当前程序运行结束时间为：Thu Oct 13 10:44:37 2022
当前正在训练第1200次， 损失是0.367170125246048
当前程序运行结束时间为：Thu Oct 13 10:45:52 2022
当前正在训练第1300次， 损失是0.32502636313438416
当前程序运行结束时间为：Thu Oct 13 1


KeyboardInterrupt



In [16]:
def train_with_constrains(): 
    model = CvxModel(n_feature=4, n_hidden=200, n_output=1)
    constraints = weightConstraint()
    criterion = torch.nn.MSELoss(reduction='sum')
    learning_rate = 1e-1  #更改了学习率 loss下降的太低了
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    terminate_threshold = 1e-3 # 1e-5
    i = 0
    print("当前程序开始运行时间为：{}".format(time.asctime(time.localtime(time.time()))))
    while (True):
        
        y_pred = model(x_train)
        loss = (criterion((y_pred).float(), (y_train).float()) / y_train.size()[0])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        model._modules['input_layer'].apply(constraints)
        model._modules['hidden_layer'].apply(constraints)
        model._modules['output_layer'].apply(constraints)
        i += 1
        if i % 100 == 0:
            print("当前程序运行结束时间为：{}".format(time.asctime(time.localtime(time.time()))))
            print("当前正在训练第{}次， 损失是{}".format(i, loss.item()))
#             print(y_train-y_pred)
            # test_model(model)
            # weight_print(model)
        if loss.item() < terminate_threshold:
            print("训练满足精度要求，训练了{}次。".format(i))
            break

train_with_constrains()

当前程序开始运行时间为：Thu Oct 13 11:36:17 2022
当前程序运行结束时间为：Thu Oct 13 11:37:35 2022
当前正在训练第100次， 损失是0.09338926523923874
当前程序运行结束时间为：Thu Oct 13 11:38:55 2022
当前正在训练第200次， 损失是0.055986445397138596
当前程序运行结束时间为：Thu Oct 13 11:40:12 2022
当前正在训练第300次， 损失是0.047510839998722076
当前程序运行结束时间为：Thu Oct 13 11:41:28 2022
当前正在训练第400次， 损失是0.046201739460229874
当前程序运行结束时间为：Thu Oct 13 11:42:43 2022
当前正在训练第500次， 损失是0.045715413987636566
当前程序运行结束时间为：Thu Oct 13 11:44:00 2022
当前正在训练第600次， 损失是0.0449470579624176
当前程序运行结束时间为：Thu Oct 13 11:45:16 2022
当前正在训练第700次， 损失是0.0439370721578598
当前程序运行结束时间为：Thu Oct 13 11:46:32 2022
当前正在训练第800次， 损失是0.043569304049015045
当前程序运行结束时间为：Thu Oct 13 11:47:49 2022
当前正在训练第900次， 损失是0.043525855988264084
当前程序运行结束时间为：Thu Oct 13 11:49:05 2022
当前正在训练第1000次， 损失是0.04318387061357498
当前程序运行结束时间为：Thu Oct 13 11:50:22 2022
当前正在训练第1100次， 损失是0.04329902306199074
当前程序运行结束时间为：Thu Oct 13 11:51:41 2022
当前正在训练第1200次， 损失是0.043030403554439545
当前程序运行结束时间为：Thu Oct 13 11:52:59 2022
当前正在训练第1300次， 损失是0.043963655829429626
当前程序

KeyboardInterrupt: 

#### 当前正在训练第300次， 损失是0.047510839998722076

In [ ]:
class CvxModel(nn.Module):
    '''
        输入凸神经网络结构定义
    '''

    def __init__(self, n_feature=4, n_hidden=4, n_output=1, nGdIter=30):
        super(CvxModel, self).__init__()
        self.input_layer = nn.Linear(n_feature, n_hidden, bias=False)
        self.hidden_layer = nn.Linear(n_hidden, n_hidden, bias=False)
        self.hidden_layer = nn.Linear(n_hidden, n_hidden, bias=False)
        self.output_layer = nn.Linear(n_hidden, n_output, bias=False)
        self.passthrough_layer = nn.Linear(n_feature, n_hidden)
        self.passthrough_output_layer = nn.Linear(n_feature, n_output)

    def forward(self, x):
        x = x.to(torch.float32)
        zx1 = F.relu(self.input_layer(x))
        pass1 = self.passthrough_layer(x)
        pass2 = self.passthrough_output_layer(x)
        zx2 = F.relu(self.hidden_layer(zx1) + pass1)
        zx3 = self.output_layer(zx2) + pass2
        zx3 = zx3.squeeze(-1)
        return zx3
改变p高

class weightConstraint():
    '''
        对一个特定的层的weight进行参数限制
    '''

    def __init__(self):
        pass

    def __call__(self, module):
        if hasattr(module, 'weight'):
            w = module.weight.data
            w = w.clamp(0.0, 100.0) #clamp函数可以设置最大最小值，
            module.weight.data = w

def train_with_constrains(): 
    model = CvxModel(n_feature=4, n_hidden=500, n_output=1)
    constraints = weightConstraint()
    criterion = torch.nn.MSELoss(reduction='sum')
    learning_rate = 1e-1  #更改了学习率 loss下降的太低了
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    terminate_threshold = 3e-3 # 1e-5
    i = 0
    print("当前程序开始运行时间为：{}".format(time.asctime(time.localtime(time.time()))))
    while (True):
        
        y_pred = model(x_train)
        loss = (criterion((y_pred).float(), (y_train).float()) / y_train.size()[0])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        model._modules['input_layer'].apply(constraints)
        model._modules['hidden_layer'].apply(constraints)
        model._modules['output_layer'].apply(constraints)
        i += 1
        if i % 100 == 0:
            print("当前程序运行结束时间为：{}".format(time.asctime(time.localtime(time.time()))))
            print("当前正在训练第{}次， 损失是{}".format(i, loss.item()))
#             print(y_train-y_pred)
            # test_model(model)
            # weight_print(model)
        if loss.item() < terminate_threshold:
            print("训练满足精度要求，训练了{}次。".format(i))
            break

train_with_constrains()

当前程序开始运行时间为：Thu Oct 13 13:24:38 2022
当前程序运行结束时间为：Thu Oct 13 13:28:55 2022
当前正在训练第100次， 损失是0.3930050730705261
当前程序运行结束时间为：Thu Oct 13 13:33:20 2022
当前正在训练第200次， 损失是0.2473527491092682
当前程序运行结束时间为：Thu Oct 13 13:38:01 2022
当前正在训练第300次， 损失是0.16390392184257507
当前程序运行结束时间为：Thu Oct 13 13:42:31 2022
当前正在训练第400次， 损失是0.1134653389453888
当前程序运行结束时间为：Thu Oct 13 13:46:45 2022
当前正在训练第500次， 损失是0.08443906903266907
当前程序运行结束时间为：Thu Oct 13 13:51:11 2022
当前正在训练第600次， 损失是0.06781134009361267
当前程序运行结束时间为：Thu Oct 13 13:55:27 2022
当前正在训练第700次， 损失是0.0578727126121521
当前程序运行结束时间为：Thu Oct 13 13:59:48 2022
当前正在训练第800次， 损失是0.05192474648356438
当前程序运行结束时间为：Thu Oct 13 14:04:10 2022
当前正在训练第900次， 损失是0.04865332692861557
当前程序运行结束时间为：Thu Oct 13 14:08:46 2022
当前正在训练第1000次， 损失是0.04667331650853157
当前程序运行结束时间为：Thu Oct 13 14:13:24 2022
当前正在训练第1100次， 损失是0.04537602886557579
当前程序运行结束时间为：Thu Oct 13 14:18:12 2022
当前正在训练第1200次， 损失是0.044472016394138336
当前程序运行结束时间为：Thu Oct 13 14:26:54 2022
当前正在训练第1400次， 损失是0.04355146363377571
当前程序运行结束时间为：T